### 파이프라인(Retriever-Reader) 평가

1. pipepath경로에 run.py의 최하단 부분에서 저장했던 파이프라인 경로를 적어줍니다.
2. readerpath경로에 reader모델(또는 run.py에서 훈련시켰던)을 저장한 경로를 적어줍니다.
3. 무지성 실행 고고

In [1]:
from haystack.schema import EvaluationResult

pipepath = '/opt/ml/input/level2_nlp_mrc-nlp-01/output/2023-06-13-11:47/pipeline_result'
readerpath = '/opt/ml/input/level2_nlp_mrc-nlp-01/output/2023-06-13-14:33/reader'

saved_eval_result = EvaluationResult.load(pipepath)

In [2]:
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.7625
Retriever - Recall (multiple relevant documents): 0.7625
Retriever - Mean Reciprocal Rank: 0.5180601722235327
Retriever - Precision: 0.04307065217391305
Retriever - Mean Average Precision: 0.46887231846536087
Reader - F1-Score: 0.5001033443680503
Reader - Exact Match: 0.4


In [3]:
from haystack.nodes import FARMReader
from haystack.nodes import BM25Retriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore()

In [5]:
retriever = BM25Retriever(document_store=document_store)
reader = FARMReader(model_name_or_path=readerpath)

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever) 

Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}


In [6]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit: 0.762
                        |
                      Reader
                        |
                        | exact_match:   0.4
                        | exact_match_top_1:   0.3
                        | f1:   0.5
                        | f1_top_1: 0.343
                        | num_examples_for_eval: 2.4e+02
                        | num_examples_for_eval_top_1: 2.4e+02
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	일본의 대학 입시는 며칠간 진행되는가?
Gold Document Ids: 
 	mrc-0-001073
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
 	ndcg: 0.0
Documents: 
 	context: 바이쿨라 전투 이후 하스드루발 바르카는 이탈리아로 떠났고 히스파니아의 카르타고군은 

In [8]:
#> SAS 지표

eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)


advanced_eval_result = pipeline.eval(
    labels=eval_labels, 
    params={"Retriever": {"top_k": 10}},
    sas_model_name_or_path="cross-encoder/stsb-roberta-large"
)

metrics = advanced_eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

...

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

0.8229997158050537
